In [19]:
import pandas as pd

In [20]:
df = pd.read_json('../data/results.jsonl', lines=True)

In [21]:
df.head()

,model_id,similarity_function,problematic_queries_count,problematic_queries,avg_embedding_time,accuracy@k,precision@k,recall@k,ndcg@k,mrr@k,map@k
0,sentence-transformers/all-MiniLM-L6-v2,cosine,11,"[""Zoom invitation on April 21, 2021"", ""Pat ask...",0,"{'1': 0.6477272727000001, '3': 0.7159090909, '...","{'1': 0.6477272727000001, '3': 0.2575757576, '...","{'1': 0.5667613636000001, '3': 0.6439393939, '...",{'10': 0.1773332362},{'10': 0.7061056999},{'100': 0.6501311296}
1,sentence-transformers/multi-qa-mpnet-base-dot-v1,cosine,8,"[""ASP.NET Core features email"", ""Zoom invitati...",0,"{'1': 0.5795454545000001, '3': 0.7954545455, '...","{'1': 0.5795454545000001, '3': 0.2878787879, '...","{'1': 0.5014204545000001, '3': 0.7050189394, '...",{'10': 0.17822341090000002},{'10': 0.6984217172},{'100': 0.6336184066}
2,thenlper/gte-base,cosine,3,"[""Photo ID pickup details for Igor Rendulic"", ...",0,"{'1': 0.75, '3': 0.8636363636000001, '5': 0.92...","{'1': 0.75, '3': 0.3181818182, '5': 0.20681818...","{'1': 0.6396780303, '3': 0.7580492424, '5': 0....",{'10': 0.2039410857},{'10': 0.8194399351},{'100': 0.7313883779}
3,intfloat/e5-small-v2,cosine,3,"[""Akontacija DDPO junij 2020"", ""Photo ID picku...",0,"{'1': 0.7159090909, '3': 0.8295454545000001, '...","{'1': 0.7159090909, '3': 0.30681818180000003, ...","{'1': 0.5942234848, '3': 0.7391098485, '5': 0....",{'10': 0.2006664699},{'10': 0.7955312049000001},{'100': 0.7082197894000001}


In [22]:
# format the output

flattened_data = {}
score_columns = ["accuracy@k", "precision@k", "recall@k", "ndcg@k", "mrr@k", "map@k"]
for col in score_columns:
    for key in df[col][0].keys():
        flattened_data[f"{col}_{key}"] = df[col].apply(lambda x: f"{x[key] * 100:.2f}%" if key in x else "0%")

flattened_df = pd.DataFrame(flattened_data)
df = pd.concat([df.drop(columns=score_columns), flattened_df], axis=1)

In [23]:
cols = list(df.columns)
sorted_metrics = [col for col in cols if any(metric in col for metric in score_columns)]
new_order = ["model_id"] + sorted_metrics + [col for col in cols if col not in sorted_metrics and col != "model_id"]
df = df[new_order]

In [24]:
df

,model_id,accuracy@k_1,accuracy@k_3,accuracy@k_5,accuracy@k_10,precision@k_1,precision@k_3,precision@k_5,precision@k_10,recall@k_1,recall@k_3,recall@k_5,recall@k_10,ndcg@k_10,mrr@k_10,map@k_100,similarity_function,problematic_queries_count,problematic_queries,avg_embedding_time
0,sentence-transformers/all-MiniLM-L6-v2,64.77%,71.59%,80.68%,87.50%,64.77%,25.76%,17.27%,10.11%,56.68%,64.39%,72.92%,80.49%,17.73%,70.61%,65.01%,cosine,11,"[""Zoom invitation on April 21, 2021"", ""Pat ask...",0
1,sentence-transformers/multi-qa-mpnet-base-dot-v1,57.95%,79.55%,86.36%,90.91%,57.95%,28.79%,18.64%,10.34%,50.14%,70.50%,76.28%,81.49%,17.82%,69.84%,63.36%,cosine,8,"[""ASP.NET Core features email"", ""Zoom invitati...",0
2,thenlper/gte-base,75.00%,86.36%,92.05%,96.59%,75.00%,31.82%,20.68%,11.14%,63.97%,75.80%,79.92%,83.71%,20.39%,81.94%,73.14%,cosine,3,"[""Photo ID pickup details for Igor Rendulic"", ...",0
3,intfloat/e5-small-v2,71.59%,82.95%,92.05%,96.59%,71.59%,30.68%,20.68%,11.25%,59.42%,73.91%,81.25%,85.84%,20.07%,79.55%,70.82%,cosine,3,"[""Akontacija DDPO junij 2020"", ""Photo ID picku...",0
